In [ ]:
!pip install torch torchvision torchaudio
!pip install datasets transformers
!pip install g2p_en praatio librosa

In [25]:
import os
from os.path import exists, join, expanduser

os.chdir(expanduser("~"))
charsiu_dir = 'charsiu'
if exists(charsiu_dir):
  !rm -rf /root/charsiu
if not exists(charsiu_dir):
  ! git clone -b development https://github.com/lingjzhu/$charsiu_dir
  ! cd charsiu && git checkout && cd -
  
os.chdir(charsiu_dir)    

Cloning into 'charsiu'...
remote: Enumerating objects: 294, done.
remote: Counting objects: 100% (294/294), done.
remote: Compressing objects: 100% (251/251), done.
remote: Total 294 (delta 137), reused 135 (delta 38), pack-reused 0
Receiving objects: 100% (294/294), 505.20 KiB | 2.05 MiB/s, done.
Resolving deltas: 100% (137/137), done.
Your branch is up to date with 'origin/development'.
/root


In [ ]:
import sys
import torch
from datasets import load_dataset
import matplotlib.pyplot as plt
sys.path.append('src/')
#sys.path.insert(0,'src')
from Charsiu import charsiu_forced_aligner, charsiu_attention_aligner

In [ ]:
timit = load_dataset('timit_asr')

In [13]:
# load data
sample = timit['train'][0]
text = sample['text']
audio_path = sample['file']
print('Text transcription:%s'%(text))
print('Audio path: %s'%audio_path)

Text transcription:Would such an act of refusal be useful?
Audio path: /root/.cache/huggingface/datasets/downloads/extracted/404950a46da14eac65eb4e2a8317b1372fb3971d980d91d5d5b221275b1fd7e0/data/TRAIN/DR4/MMDM0/SI681.WAV


In [20]:
# initialize model
charsiu = charsiu_forced_aligner(aligner='charsiu/en_w2v2_fc_10ms')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:341: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


Forced alignment with a neural forced alignment model

In [21]:
alignment = charsiu.align(audio=audio_path,text=text)

In [22]:
print(alignment)
print('\n Ground Truth \n')
print([(s/16000,e/16000,p) for s,e,p in zip(sample['phonetic_detail']['start'],sample['phonetic_detail']['stop'],sample['phonetic_detail']['utterance'])])

[(0.0, 0.08, '[SIL]'), (0.08, 0.15, 'W'), (0.15, 0.19, 'UH'), (0.19, 0.24, 'D'), (0.24, 0.38, 'S'), (0.38, 0.46, 'AH'), (0.46, 0.58, 'CH'), (0.58, 0.6, 'AE'), (0.6, 0.68, 'N'), (0.68, 0.82, 'AE'), (0.82, 0.93, 'K'), (0.93, 0.99, 'T'), (0.99, 1.04, 'AH'), (1.04, 1.13, 'V'), (1.13, 1.17, 'R'), (1.17, 1.22, 'AH'), (1.22, 1.33, 'F'), (1.33, 1.41, 'Y'), (1.41, 1.48, 'UW'), (1.48, 1.53, 'Z'), (1.53, 1.62, 'AH'), (1.62, 1.68, 'L'), (1.68, 1.78, 'B'), (1.78, 1.88, 'IY'), (1.88, 1.99, 'Y'), (1.99, 2.08, 'UW'), (2.08, 2.13, 'S'), (2.13, 2.23, 'F'), (2.23, 2.27, 'AH'), (2.27, 2.47, 'L'), (2.47, 2.48, '[SIL]')]

 Ground Truth 

[(0.0, 0.1225, 'h#'), (0.1225, 0.154125, 'w'), (0.154125, 0.2175, 'ix'), (0.2175, 0.25, 'dcl'), (0.25, 0.3725, 's'), (0.3725, 0.4675, 'ah'), (0.4675, 0.4925, 'tcl'), (0.4925, 0.5875, 'ch'), (0.5875, 0.6225, 'ix'), (0.6225, 0.6675, 'n'), (0.6675, 0.8425, 'ae'), (0.8425, 0.98, 'kcl'), (0.98, 0.9925, 't'), (0.9925, 1.0575, 'ix'), (1.0575, 1.1435625, 'v'), (1.1435625, 1.180125,

In [23]:
# save alignment
charsiu.serve(audio=audio_path,text=text,save_to='./local/sample.TextGrid')

Alignment output has been saved to ./local/sample.TextGrid


Forced Alignment with An Attention Alignment Model

In [26]:
charsiu = charsiu_attention_aligner('charsiu/en_w2v2_fs_10ms')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:341: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


AttributeError: ignored

In [ ]:
# load data
sample = timit['train'][0]
text = sample['text']
audio_path = sample['file']
print('Text transcription:%s'%(text))
print('Audio path: %s'%audio_path)

Text transcription:Would such an act of refusal be useful?
Phone transcription: ['W', 'UH', 'D', ' ', 'S', 'AH', 'CH', ' ', 'AE', 'N', ' ', 'AE', 'K', 'T', ' ', 'AH', 'V', ' ', 'R', 'AH', 'F', 'Y', 'UW', 'Z', 'AH', 'L', ' ', 'B', 'IY', ' ', 'Y', 'UW', 'S', 'F', 'AH', 'L', ' ', '?']


In [ ]:
alignment = charsiu.align(audio=audio_path,text=text)

In [ ]:
print(alignment)
print('\n Ground Truth \n')
print([(s/16000,e/16000,p) for s,e,p in zip(sample['phonetic_detail']['start'],sample['phonetic_detail']['stop'],sample['phonetic_detail']['utterance'])])

[(0.0, 0.08, '[SIL]'), (0.08, 0.15, 'W'), (0.15, 0.19, 'UH'), (0.19, 0.24, 'D'), (0.24, 0.38, 'S'), (0.38, 0.46, 'AH'), (0.46, 0.58, 'CH'), (0.58, 0.6, 'AE'), (0.6, 0.68, 'N'), (0.68, 0.82, 'AE'), (0.82, 0.93, 'K'), (0.93, 0.99, 'T'), (0.99, 1.04, 'AH'), (1.04, 1.13, 'V'), (1.13, 1.17, 'R'), (1.17, 1.22, 'AH'), (1.22, 1.33, 'F'), (1.33, 1.41, 'Y'), (1.41, 1.48, 'UW'), (1.48, 1.53, 'Z'), (1.53, 1.62, 'AH'), (1.62, 1.68, 'L'), (1.68, 1.78, 'B'), (1.78, 1.88, 'IY'), (1.88, 1.99, 'Y'), (1.99, 2.08, 'UW'), (2.08, 2.13, 'S'), (2.13, 2.23, 'F'), (2.23, 2.27, 'AH'), (2.27, 2.47, 'L'), (2.47, 2.48, '[SIL]')]
\n Ground Truth \n
[(0.0, 0.1225, 'h#'), (0.1225, 0.154125, 'w'), (0.154125, 0.2175, 'ix'), (0.2175, 0.25, 'dcl'), (0.25, 0.3725, 's'), (0.3725, 0.4675, 'ah'), (0.4675, 0.4925, 'tcl'), (0.4925, 0.5875, 'ch'), (0.5875, 0.6225, 'ix'), (0.6225, 0.6675, 'n'), (0.6675, 0.8425, 'ae'), (0.8425, 0.98, 'kcl'), (0.98, 0.9925, 't'), (0.9925, 1.0575, 'ix'), (1.0575, 1.1435625, 'v'), (1.1435625, 1.18012

In [ ]:
charsiu.serve(audio=audio_path,text=text,save_to='./local/sample.TextGrid')